## Importuri

In [1]:
import os
from pathlib import Path
import torch
from yolov5 import train
from yolov5 import detect
import yaml

## Functia pentru antrenare

In [13]:
# Setează parametrii pentru antrenare
def train_yolov5():
    # Calea către fișierul de configurare
    dataset_yaml = "CaltechPedestrian.yaml"
    img_size = 640
    batch_size = 10
    epochs = 10
    device = 0

    # Hiperparametrii au fost copiati din fisierul ./yolov5/data/hyps/hyp.scratch-low.yaml
    # S-a modificat doar learning rate-ul, deoarece antrenarea dureaza aproximativ 11 ore pentru 10 epoci
    # si se doreste ca algoritmul sa convearga mai repede
    hyp = {
        'lr0': 0.03,                # initial learning rate (SGD=1E-2, Adam=1E-3)
        'lrf': 0.01,                # final OneCycleLR learning rate (lr0 * lrf)
        'momentum': 0.937,          # SGD momentum/Adam beta1
        'weight_decay': 0.0005,     # optimizer weight decay 5e-4
        'warmup_epochs': 3.0,       # warmup epochs (fractions ok)
        'warmup_momentum': 0.8,     # warmup initial momentum
        'warmup_bias_lr': 0.1,      # warmup initial bias lr
        'box': 0.05,                # box loss gain
        'cls': 0.5,                 # cls loss gain
        'cls_pw': 1.0,              # cls BCELoss positive_weight
        'obj': 1.0,                 # obj loss gain (scale with pixels)
        'obj_pw': 1.0,              # obj BCELoss positive_weight
        'iou_t': 0.20,              # IoU training threshold
        'anchor_t': 4.0,            # anchor-multiple threshold
        # anchors: 3,               # anchors per output layer (0 to ignore)
        'fl_gamma': 0.0,            # focal loss gamma (efficientDet default gamma=1.5)
        'hsv_h': 0.015,             # image HSV-Hue augmentation (fraction)
        'hsv_s': 0.7,               # image HSV-Saturation augmentation (fraction)
        'hsv_v': 0.4,               # image HSV-Value augmentation (fraction)
        'degrees': 0.0,             # image rotation (+/- deg)
        'translate': 0.1,           # image translation (+/- fraction)
        'scale': 0.5,               # image scale (+/- gain)
        'shear': 0.0,               # image shear (+/- deg)
        'perspective': 0.0,         # image perspective (+/- fraction), range 0-0.001
        'flipud': 0.0,              # image flip up-down (probability)
        'fliplr': 0.5,              # image flip left-right (probability)
        'mosaic': 1.0,              # image mosaic (probability)
        'mixup': 0.0,               # image mixup (probability)
        'copy_paste': 0.0           # segment copy-paste (probability)
    }

    # Salvez hiperparametrii intr-un fișier YAML temporar, pentru ca train.run() va incerca sa valideze existenta lui
    with open('./yolov5/data/hyps/hyp_custom.yaml', 'w') as f:
        yaml.dump(hyp, f)

    hyp_yaml = "./yolov5/data/hyps/hyp_custom.yaml"

    # Comandă de antrenare
    train.run(
        data=dataset_yaml,                 # Setul de date
        weights='',                        # Fără greutăți pre-antrenate
        cfg='yolov5n.yaml',                # Arhitectura modelului YOLOv5n
        epochs=epochs,                     # Număr de epoci
        batch_size=batch_size,             # Dimensiunea batch-ului
        device=device,                     # Utilizează GPU-ul specificat
        imgsz=img_size,                    # Dimensiunea imaginilor
        hyp=hyp_yaml,                      # Hiperparametrii personalizați
        project='./yolov5/runs/train',     # Folderul unde se salvează rezultatele
        name='exp',                       # Numele experimentului
        exist_ok=True                      # Suprascrie dacă există deja rezultatele
    )


In [14]:
train_yolov5()

train: weights=, cfg=yolov5n.yaml, data=CaltechPedestrian.yaml, hyp=./yolov5/data/hyps/hyp_custom.yaml, epochs=10, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=./yolov5/runs/train, name=exp2, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: ⚠️ YOLOv5 is out of date by 1 commit. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.8.15 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3799MiB)

hyperparameters: anchor_t=4.0, box=0.05, cls=0.5, 

KeyboardInterrupt: 

## Functia pentru testare

In [2]:
def detect_yolov5():
    # Parametrii pentru detectare
    weights_path = "./yolov5/runs/train/exp/weights/best.pt"     # Greutățile modelului antrenat
    source = "./dataset/images/test"                             # Directorul cu imaginile pentru detectare
    device = 0                                                   # GPU utilizat (0 pentru primul GPU)
    img_size = (640, 640)                                        # Dimensiunea imaginilor
    conf_thres = 0.25                                            # Pragul de încredere
    iou_thres = 0.45                                             # Pragul IoU
    save_txt = True                                              # Salvează rezultatele în fișiere TXT
    save_conf = True                                             # Salvează și scorurile de încredere
    save_crop = False                                            # Nu salvează decupajele detectate
    project = './yolov5/runs/detect'                             # Directorul pentru salvarea rezultatelor
    name = 'exp'                                                 # Numele experimentului
    exist_ok = True                                              # Suprascrie rezultatele existente

    # Execută detectarea
    detect.run(
        weights=weights_path,    # Greutățile modelului YOLOv5
        source=source,           # Directorul sau fișierul sursă
        device=device,           # Utilizează GPU-ul specificat
        imgsz=img_size,          # Dimensiunea imaginilor
        conf_thres=conf_thres,   # Pragul de încredere
        iou_thres=iou_thres,     # Pragul IoU
        save_txt=save_txt,       # Salvează rezultatele în fișiere TXT
        save_conf=save_conf,     # Salvează scorurile de încredere
        save_crop=save_crop,     # Nu decupează obiectele detectate
        project=project,         # Directorul pentru salvarea rezultatelor
        name=name,               # Numele experimentului
        exist_ok=exist_ok        # Suprascrie rezultatele existente
    )


In [3]:
detect_yolov5()

YOLOv5 🚀 v7.0-388-g882c35fc Python-3.8.15 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 3799MiB)

Fusing layers... 
YOLOv5n summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
image 1/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_0.jpg: 480x640 (no detections), 57.6ms
image 2/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_1.jpg: 480x640 (no detections), 9.8ms
image 3/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_10.jpg: 480x640 (no detections), 9.5ms
image 4/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_100.jpg: 480x640 (no detections), 8.6ms
image 5/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_1000.jpg: 480x640 (no detections), 9.3ms
image 6/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_1001.jpg: 480x640 (no detections), 8.9ms
image 7/44263 /home/alex/Documents/SCCV/proiect/dataset/images/test/set09_V000_1002

KeyboardInterrupt: 